<a href="https://colab.research.google.com/github/samherreid/See-G/blob/main/See_G.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## See-G(IS code)

By Sam Herreid 

October 2022

See-G is a tool for visual learners to see arcpy functions plotted in space and turned into a sequential video to troubleshoot, vision board, identify inefficiencies and communicate many-step methods quickly and exhaustively in talks and paper supplemental information.

In order to use See-G, you have to set up an ArcGIS Pro project with a layout following https://pro.arcgis.com/en/pro-app/latest/arcpy/mapping/tutorial-getting-started-with-arcpy-mp.htm

Call the below function in the inline example below.

In [ ]:
baseMap = r"" # .shp (optional)
arcGisProject = r"" # .aprx
frameFolderPath = r""

In [ ]:
imageCount = 0 # inititalize (or reset) the count

In [ ]:
import arcpy

In [ ]:
baseMapLayer = arcpy.MakeFeatureLayer_management(baseMap, "base_lyr").getOutput(0)

In [ ]:
def imgCount():
  imageCount = globals()['imageCount'] + 1
  globals()['imageCount'] = imageCount
  return str(globals()['imageCount']).zfill(4)

In [ ]:
def makeFrame(featureToAdd, arcGisProject, baseMapLayer, frameFolderPath):
  aprx = arcpy.mp.ArcGISProject(arcGisProject)
  m = aprx.listMaps("Map")[0]
  feat_lyr = arcpy.MakeFeatureLayer_management(featureToAdd, "feat_lyr").getOutput(0)
  map_lyr = m.addLayer(baseMapLayer)[0]
  map_lyr = m.addLayer(feat_lyr, "TOP")[0]
  lyr = m.listLayers(feat_lyr)[0]
  lyr_base = m.listLayers(baseMapLayer)[0]

  # set base map layer color
  sym = lyr_base.symbology
  colorRamp = aprx.listColorRamps("Cyan to Purple")[0]
  sym.renderer.colorRamp = colorRamp
  lyr_base.symbology = sym

  lyt = aprx.listLayouts()[0]
  mf = lyt.listElements("mapframe_element")[0]
  mf.camera.setExtent(mf.getLayerExtent(lyr, False, True))
  lyt.exportToPNG(frameFolderPath+"\\frame"+imgCount()+".png")
  arcpy.Delete_management("feat_lyr")
  del feat_lyr, map_lyr, lyr, lyt, mf

With the above setup and functons imported into your code, use this conditional statement after any output of interest is generated (particularly intermediate steps where the files are deleted) to save a frame. The if statment allows the option of printing frames or not, since it will slow down your code and you probably only need to make a methods video every once in a while. 

In [ ]:
makeFrames = 'y' 

arcpy.someFunction(input,output) # I think only .shp works right now, but grids should be easy to accomodate

# See-G code
if makeFrames == 'y':
  makeFrame(output, arcGisProject, baseMapLayer, frameFolderPath)
  # only output should change, the other three variables will be the same unless you want to change the base map layer

A next addition will be adding text to the frames about the function called and the line number